In [1]:
from training_rl.offline_rl.custom_envs.utils import Grid2DInitialConfig, InitialConfigCustom2DGridEnvWrapper
from training_rl.offline_rl.custom_envs.custom_2d_grid_env.obstacles_2D_grid_register import ObstacleTypes
from training_rl.offline_rl.custom_envs.custom_envs_registration import CustomEnv, RenderMode
from training_rl.offline_rl.custom_envs.custom_envs_registration import register_grid_envs
import warnings
from training_rl.offline_rl.load_env_variables import load_env_variables
import gymnasium as gym
from training_rl.offline_rl.offline_policies.policy_registry import PolicyName
from training_rl.offline_rl.behavior_policies.behavior_policy_registry import BehaviorPolicyType
from training_rl.offline_rl.generate_custom_minari_datasets.generate_minari_dataset_grid_envs import \
    create_combined_minari_dataset
from training_rl.offline_rl.offline_policies.offpolicy_rendering import offpolicy_rendering
from training_rl.offline_rl.offline_trainings.policy_config_data_class import TrainedPolicyConfig
from training_rl.offline_rl.offline_trainings.offline_training import offline_training
import torch
from training_rl.offline_rl.offline_trainings.policy_config_data_class import get_trained_policy_path
import os
from training_rl.offline_rl.offline_trainings.restore_policy_model import restore_trained_offline_policy


load_env_variables()

warnings.filterwarnings("ignore")
# ToDo: this should be load automatically
register_grid_envs()


/home/ivan/anaconda3/envs/tianshou_dev/lib/python3.8/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment GymV26Environment-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/ivan/anaconda3/envs/tianshou_dev/lib/python3.8/site-packages/google/rpc/__init__.py:20: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.rpc')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  pkg_resources.declare_namespace(__name__)
/home/ivan/anaconda3/envs/tianshou_dev/lib/python3.8/site-packages/pkg_resources/__init__.py:2350: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`

**Before we saw that the DQN algorithm was bad to connect pieces of trajectories coming from different datasets (stiching). Here we will see how some of the offline algorithms introduce before, namely the BCQ and the CQL, handle this problem.**

We will start again with the previous setup. So as we did before we will create again two datasets one from a policy moving suboptimal from (0,0) to (7,0) and the other from another policy moving from (4,0) to (7,7). 

### Environment

In [2]:
ENV_NAME = CustomEnv.Grid_2D_8x8_discrete

# Env. Config.
OBSTACLE = ObstacleTypes.verical_object_8x8
INITIAL_STATE = (0, 0)
FINAL_STATE = (7, 7)

env_2D_grid_initial_config = Grid2DInitialConfig(
    obstacles=OBSTACLE,
    initial_state=INITIAL_STATE,
    target_state=FINAL_STATE,
)

env = InitialConfigCustom2DGridEnvWrapper(gym.make(ENV_NAME, render_mode=RenderMode.RGB_ARRAY_LIST), 
                                          env_config=env_2D_grid_initial_config)

### Configure the two datasets

In [3]:
IDENTIFIER_COMBINED_DATASETS = "_stiching_property_I"

# Dataset I
BEHAVIOR_POLICY_I = BehaviorPolicyType.behavior_8x8_moves_downwards_within_strip
DATA_SET_IDENTIFIER_I = "_move_downwards"
NUM_STEPS_I = 3000

# Dataset II
#BEHAVIOR_POLICY_II = BehaviorPolicyType.behavior_8x8_eps_greedy_4_0_to_7_7
BEHAVIOR_POLICY_II = BehaviorPolicyType.behavior_8x8_deterministic_4_0_to_7_7
DATA_SET_IDENTIFIER_II = "_move_deterministic"
NUM_STEPS_II = 300


# Policy to be trained
OFFLINE_POLICY = PolicyName.bcq_discrete

### Create Minari combined dataset

In [4]:
config_combined_data = create_combined_minari_dataset(
        env_name=ENV_NAME,
        dataset_identifiers = (DATA_SET_IDENTIFIER_I, DATA_SET_IDENTIFIER_II),
        num_colected_points = (NUM_STEPS_I, NUM_STEPS_II),
        behavior_policy_names = (BEHAVIOR_POLICY_I, BEHAVIOR_POLICY_II),
        combined_dataset_identifier = "combined_dataset",
        env_2d_grid_initial_config = env_2D_grid_initial_config,
)

Dataset Grid_2D_8x8_discrete-data_verical_object_8x8_start_0_0_target_7_7_move_downwards-V0 deleted!
Dataset Grid_2D_8x8_discrete-data_verical_object_8x8_start_0_0_target_7_7_move_deterministic-V0 deleted!
Dataset Grid_2D_8x8_discrete-combined_dataset-V0 deleted!
Number of episodes in dataset I:50, in dataset I:70 and  in the combined dataset: 120


### Rendering behavioral policy 

EXERCISE:

I - Train the following algorithms: CQL, BCQ and DQN
II - Compare the results and extract conclusions.

(
DQN --> BAD
CQL --> better but not optimal
BCQ --> better 
)

III - increase the size of data II

What do you observe?


In [5]:

# Suboptimal policy
offpolicy_rendering(
    env_or_env_name=ENV_NAME,
    render_mode=RenderMode.RGB_ARRAY_LIST,
    behavior_policy_name=BEHAVIOR_POLICY_II,
    env_2d_grid_initial_config=env_2D_grid_initial_config,
    num_frames=1000,
)

QObject::moveToThread: Current thread (0x5576b0e424d0) is not the object's thread (0x5576b0c397d0).
Cannot move to target thread (0x5576b0e424d0)

QObject::moveToThread: Current thread (0x5576b0e424d0) is not the object's thread (0x5576b0c397d0).
Cannot move to target thread (0x5576b0e424d0)

QObject::moveToThread: Current thread (0x5576b0e424d0) is not the object's thread (0x5576b0c397d0).
Cannot move to target thread (0x5576b0e424d0)

QObject::moveToThread: Current thread (0x5576b0e424d0) is not the object's thread (0x5576b0c397d0).
Cannot move to target thread (0x5576b0e424d0)

QObject::moveToThread: Current thread (0x5576b0e424d0) is not the object's thread (0x5576b0c397d0).
Cannot move to target thread (0x5576b0e424d0)

QObject::moveToThread: Current thread (0x5576b0e424d0) is not the object's thread (0x5576b0c397d0).
Cannot move to target thread (0x5576b0e424d0)

QObject::moveToThread: Current thread (0x5576b0e424d0) is not the object's thread (0x5576b0c397d0).
Cannot move to tar

InterruptedError: You quit ('q') the animation.

### Choose yuor policy

In [6]:
# The model policy to be trained.
POLICY_NAME = PolicyName.cql_discrete


NAME_EXPERT_DATA = config_combined_data.data_set_name
# TrainedPolicyConfig is a handy object that will help us to deal with the policy configuration data.
offline_policy_config = TrainedPolicyConfig(
    name_expert_data=NAME_EXPERT_DATA,
    policy_name=POLICY_NAME,
    render_mode=RenderMode.RGB_ARRAY_LIST,
    device="cpu",
)

### Training ( If you want to use an already trained policy run the cell below and skip this one !! )

In [ ]:
NUM_EPOCHS = 50
BATCH_SIZE = 256
UPDATE_PER_EPOCH = 100

# After every epoch we will collect some test statistics from the policy from NUMBER_TEST_ENVS independent envs.
NUMBER_TEST_ENVS = 1
EXPLORATION_NOISE = True
SEED = None #1626


# Run the training
offline_training(
    offline_policy_config=offline_policy_config,
    num_epochs = NUM_EPOCHS,
    number_test_envs=NUMBER_TEST_ENVS,
    update_per_epoch=UPDATE_PER_EPOCH,
    restore_training=False,
)

In [8]:
POLICY_FILE = "policy_final.pth"
# restore a policy with the same configuration as the one we trained.
policy = restore_trained_offline_policy(offline_policy_config)
# load the weights
name_expert_data = offline_policy_config.name_expert_data
log_name = os.path.join(name_expert_data, POLICY_NAME)
log_path = get_trained_policy_path(log_name)
policy.load_state_dict(torch.load(os.path.join(log_path, POLICY_FILE), map_location="cpu"))


<All keys matched successfully>

In [ ]:
offpolicy_rendering(
    env_or_env_name=env,
    render_mode=RenderMode.RGB_ARRAY_LIST,
    policy_model=policy,
    env_2d_grid_initial_config=env_2D_grid_initial_config,
    num_frames=1000,
    imitation_policy_sampling=False
)

### Exercise 2

How to improve CQL?? --> change params, more data?
???